# **2022 ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Training config: mini-batch=512, optimizer=Adam, learning rate=0.1 (TODO: Change the config!)



# **Import Some Packages**

In [111]:
import numpy as np
import csv
import math
import pandas as pd
import matplotlib.pyplot as plt

In [112]:
import os
os.getcwd()

'd:\\NTU\\First Year\\Machine Learning\\HW\\HW1\\Programming'

# **Fix random seed**


This is for the reproduction of your result. **DO NOT modify this secton!** 


In [113]:
import random

seed = 9487
random.seed(seed)

# **Download training data**


In [114]:
# !gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"s
# !gdown --id '1FXJztppYG9Q4b_4NHvcmPsc4o5obceWC' --output "test.csv"

# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
# !gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content

# **Define function**

In [115]:
def valid(x):
    # Total unvalid datas is 35
    global count
    dic = {
        'AMB_TEMP':0, 'CO':1, 'NO':2, 'NO2':3, 'NOx':4, 'O3':5,
        'PM10':6, 'WS_HR':7, 'RAINFALL':8, 'RH':9, 'SO2':10,
        'WD_HR':11, 'WIND_DIREC':12, 'WIND_SPEED':13, 'PM2.5':14}
    for i in range(len(x[0])):
        if x[dic['CO']][i] > 1.4 or x[dic['NO']][i] > 40 or x[dic['NO2']][i] > 55 or \
            x[dic['NOx']][i] > 75 or x[dic['O3']][i] > 120 or x[dic['PM10']][i] > 150 or \
            x[dic['RAINFALL']][i] > 30 or x[dic['SO2']][i] > 20 or x[dic['PM2.5']][i] > 65:
            count.append(i)
    count.reverse()
    for i in range(len(count)):
        for j in range(15):
            del x[j][count[i]]
    return np.array(x)

# Create your dataset
def parse2train_x(data, feats):

    x = []

    # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
    total_length = data.shape[1] - 8  # data.shape[1] == 5774
    
    for i in range(total_length):
        x_tmp = data[feats, i:i+8] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc. Feats is row of data and i:i+8 is column of data
        x.append(x_tmp.reshape(-1,))
    
    # Total valid datas is 5739
    # x.shape: (n, 15, 8)
    x = np.array(x)
    return x

def parse2train_y(data, feats):
    y = []

    # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
    total_length = data.shape[1] - 8  # data.shape[1] == 5774
    
    for i in range(total_length):
        y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5
        y.append(y_tmp)
    
    y = np.array(y)
    return y
    
def parse2test(data, feats):
    x = []
    for i in range(90):
        x_tmp = data[feats, 8*i:8*i+8]
        x.append(x_tmp.reshape(-1,))

    # x.shape: (n, 15, 8)
    x = np.array(x)
    return x

def norm_data(train_data, norm_case, test_data):
    dic_data_round = {
        0 : 1, 1 : 2, 2 : 1, 3 : 1, 4 : 1,
        5 : 1, 6 : 0, 7 : 1, 8 : 1, 9 : 0, 
        10 : 1, 11 : 0, 12 : 0, 13 : 1, 14 : 0
    }

    # Concate data
    concate_data = np.concatenate((train_data, test_data), axis=1)

    # Z-Score
    if norm_case == 0:
        mean_arr = np.zeros(15)
        std_arr = np.zeros(15)

        for i in range(15):
            # Compute mean
            mean_temp = sum(concate_data[i])
            mean_arr[i] = mean_temp / float(len(concate_data[i]))
        
            # Compute std
            std_temp = 0
            for j in range(len(concate_data[i])):
                std_temp += (concate_data[i][j] - mean_arr[i])**2
            std_arr[i] = (std_temp / float((len(concate_data[i] - 1))))**0.5

            # Create normalize data
            train_data[i] -= mean_arr[i]
            train_data[i] = np.round(train_data[i] / std_arr[i], dic_data_round[i]+1)
            test_data[i] -= mean_arr[i]
            test_data[i] = np.round(test_data[i] / std_arr[i], dic_data_round[i]+1)
        return train_data, test_data

    # Max-Min
    elif norm_case == 1:
        for i in range(15):
            max_temp = max(concate_data[i])
            min_temp = min(concate_data[i])
            train_data[i] = np.round((train_data[i] - min_temp) / (max_temp - min_temp), dic_data_round[i]+1)
            test_data[i] = np.round((test_data[i] - min_temp) / (max_temp - min_temp), dic_data_round[i]+1)
        return train_data, test_data
    
    # MaxAbs
    elif norm_case == 2:
        for i in range(15):
            maxabs_temp = abs(max(concate_data[i]))
            train_data[i] = np.round(train_data[i] / maxabs_temp, dic_data_round[i]+1)
            test_data[i] = np.round(test_data[i] / maxabs_temp, dic_data_round[i]+1)
        return train_data, test_data

    # RobustScaler
    elif norm_case == 3:

        return train_data

    else:
        print('No define the normalize method, please check again')
        return train_data

def visul_data(data, path, norm_type=0):
    dic_data_title = {
        0 : 'AMB_TEMP', 1 : 'CO', 2 : 'NO', 3 : 'NO2', 4 : 'NOx',
        5 : 'O3', 6 : 'PM10', 7 : 'WS_HR', 8 : 'RAINFALL', 9 : 'RH',
        10 : 'SO2', 11 : 'WD_HR', 12 : 'WIND_DIREC', 13 : 'WIND_SPEED', 14 : 'PM2.5'
    }
    dic_norm_type = {
        0:'_zscore_norm.png',
        1:'_maxmin_norm.png',
        2:'_maxabs_norm.png',
        3:'_robust_norm.png'
    }

    for i in range(15):
        j = np.arange(len(data[i]))
        plt.scatter(j, data[i])
        plt.xlabel('i-th data')
        plt.ylabel(dic_data_title[i])
        plt.savefig(path + dic_data_title[i] + dic_norm_type[norm_type])
        plt.show()


In [116]:
# TODO: Implement 2-nd polynomial regression version for the report.
def minibatch(x, y, config):
  
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    # Linear regression: only contains two parameters (w, b).
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    # Training loop
    total_loss = np.zeros(epoch)
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b * batch_size:(b+1) * batch_size]
            y_batch = y[b * batch_size:(b+1) * batch_size].reshape(-1,1)

            # Prediction of linear regression 
            pred = np.dot(x_batch, w) + bias
            # loss
            loss = y_batch - pred
            # total_loss[num] += (loss**2).sum(axis=0)
            
            # Compute gradient
            g_t = np.dot(x_batch.transpose(), loss) * (-2) +  2 * lam * np.sum(w) 
            g_t_b = loss.sum(axis=0) * (-2)
            m_t = beta_1 * m_t + (1-beta_1) * g_t 
            v_t = beta_2 * v_t + (1-beta_2) * np.multiply(g_t, g_t)
            m_cap = m_t / (1-(beta_1**t))
            v_cap = v_t / (1 - (beta_2**t))
            m_t_b = 0.9 * m_t_b + (1 - 0.9) * g_t_b
            v_t_b = 0.99 * v_t_b + (1 - 0.99) * (g_t_b * g_t_b) 
            m_cap_b = m_t_b / (1 - (0.9**t))
            v_cap_b = v_t_b / (1 - (0.99**t))
            w_0 = np.copy(w)
            
            # Update weight & bias
            w -= ((lr * m_cap) / (np.sqrt(v_cap) + epsilon)).reshape(-1, 1)
            bias -= (lr * m_cap_b) / (math.sqrt(v_cap_b) + epsilon)
            
        # total_loss[num] /= x.shape[0]
        
    # plt.plot(total_loss)    
    # plt.show()
    return w, bias

In [117]:
from argparse import Namespace

# TODO: Tune the config to boost your performance. 
train_config = Namespace(
    batch_size = 512,
    lr = 1e-1,
    lam = 0.001,
    epoch = 100,
)

# **Training your regression model**

In [118]:
# Choose your features to train. 
# Hint: 
# 1. You can select more than one feature.
# 2. You should select "good" features.

# TODO: Carefully justify which feature should be chosen.
# feats = [1, 2, 3, 4, 6, 14] # Choose CO, NO, NO2, NOx, PM2.5, PM10
# feats = [0, 5, 7, 8, 9, 10, 11, 12, 13]
# feats = np.ndarray.tolist(np.arange(15))
feats = [1, 2, 3, 4, 6, 7, 8, 9, 13, 14]

In [119]:
# Data preprocessing.
train_data = pd.read_csv("./dataset/train.csv")
train_data = train_data.values  # Data type is <class 'numpy.ndarray'>

# To transpose the data to data^T
train_data = np.transpose(np.array(np.float64(train_data)))   # Data type is <class 'numpy.ndarray'>

# Filter valid train data
count = []
train_data = np.ndarray.tolist(train_data)
train_data = valid(train_data)

# Split train_y data before normalization
train_x = parse2train_x(train_data, feats)
train_y = parse2train_y(train_data, feats)


# Normalize data
# train_data, test_data_ = norm_data(train_data, 0, test_data)


test_data = pd.read_csv('./dataset/test.csv')
test_data = test_data.values
test_data = np.transpose(np.array(np.float64(test_data)))
test_x = parse2test(test_data, feats)

# **Adam**
* This is our gradient descent algorithm. Adam was implemented.
* You can implement another algorithm such as SGD, which may (or may not) boost the performance.
* However, **modules like sklearn and pytorch are not allowed**.
* Ref: https://arxiv.org/pdf/1412.6980.pdf
![](https://i.imgur.com/jRaebdf.png)



In [120]:
# Train your regression model
w, bias = minibatch(train_x, train_y, train_config)
print(w.shape, bias.shape)

(88, 1) (1,)


# **Testing & Write result as .csv**

---



In [121]:
with open('./testing_result/my_sol.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])

    print(test_x.shape) 
    for i in range(int(test_x.shape[0])):
      # Prediction of linear regression 
      prediction = (np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
      writer.writerow([i, prediction] )

(90, 88)
